In the regression setting, the standard linear model  

$$Y = \beta_0 + \beta_1X_1 + ... + \beta_pX_p + \epsilon$$  

is commonly used to describe the relationship between a response $Y$ and a set of variables $X_1, X_2, ..., X_p$. In this chapter we will discuss some ways in which the simple linear model can be improved, by replacing plain least squares fitting with some alternative fitting procedures.  

Why would we want to use another fitting procedure instead of least squares?  
 1. *Prediction Accuracy*  
  * Provided that the true relationship between the response and the predictors is approximately linear, the least squares estimates will have low bias. If $n\gg p$, then the least squares estimates tend to also have low variance, and will perform well on test observations. However if $n$ is not much larger than $p$, then there can be a lot of variability in the least squares fit, resulting in overfitting and poor predictions on future observations. And if $p > n$, then there is no longer a unique least squares coefficient estimate (The variance is *infinite* so the method cannot be used at all). By *constraining* or *shrinking* the estimated coefficients, we can ofen substantially reduce the variance at the cost of a negligible increase in bias, which can lead to substantial improvements in the accuracy of predictions.  
  <br>
 2. *Model Interpretability*
  * In many cases, some or many of the variables are not actually associated with the response. Including such *irrelevant* variables leads to unnecesary complexity in the resulting model. By removing these variables (setting their coefficients to zero), we can obtain a model that is more easily interpreted. In this case, least squares is extremely unlikely to yield any coefficient estimates that are exactly zero.  
  
In this chapter we discuss three important classes of alternatives to using least squares fit:  

 1. *Subset Selection*  
  * Identify a subset of the $p$ predictors that we believe are related to the response, then fit a model using least squares on the reduced set of variables.  
  <br>
 2. *Shrinkage*  
  * Fit a model with all $p$ predictors but shrink coefficients towards zero relative to the least squares estimates. This shrinkage (a.k.a. *regularization*) has the effect of reducing variance. Depending on what type of shrinkage is performed, some of the coefficients may be estimated to be exactly zero. Hence, shrinkage methods can also perform variable selection.  
  <br>
 3. *Dimension Reduction*  
  * This approach involves *projecting* the $p$ predictors into a $M$-dimensional space, where $M<p$. This is achieved by computing $M$ different *linear combinations* or *projections* of the variables. Then these $M$ projections are used as predictors to fit a linear regression model by least squares.  
  


# Subset Selection

## Best Subset Selection  
To perform *best subset selection* we fit a separate least squares regression for each possible combination of the $p$ predictors and then identify which one is the *best*. The problem of selecting the *best model* from the $2^p$ possibilities considered by best subset selection is not trivial. This is usually broken up into two stages, as in Algorithm 6.1.  

---
**Algorithm 6.1**: *Best subset selection*  

  1. Let $M_0$ denote the *null model*, which contains no predictors. This model simply predicts the sample mean for each observation.  
  <br>
  2. For $k=1, 2, ..., p$:  
    a. Fit all $\bigl(_k^p\bigr)$ models that contain exactly $k$ predictions  
    b. Pick the best among these $\bigl(_k^p\bigr)$ models and call it $M_k$. Here *best* is defined as having the smallest RSS, or equivalently largest $R^2$.  
  <br>
  3. Select a single best model from among $M_0, ..., M_p$ using cross validated prediction error, $C_p$ (AIC), BIC, or adjusted $R^2$.

---  

In Algorithm 6.1, step 2 identifies the best model for each subset size (on the training data) which reduces the problem from one of $2^p$ possible models to one of $p+1$ possible models.  

Now in order to select a single best model, we must simple choose among these $p+1$ options. However, since RSS decreases monotonically and $R^2$ increases monotonically as the number of features increases, we will always end up with a model involving all the variables. The problem is that a low RSS and a high $R^2$ indicates a model with a low *training* error, whereas we wish to choose a model that has a low *test* error. Therefore, in step 3, we us cross-validated prediction error, $C_p$, BIC, or adjusted $R^2$ in order to select among $M_0, M_1, ..., M_p$.  

In the case of logistic regression, *deviance* plays the role of RSS in step 2 for selecting the best model. Deviance is negative two times the maximized log-likelihood. The smaller the deviance, the better the fit.  

While best subset selection is simple and conceptually appealing, it suffers from computational limitations. If $p=10$ there are approximately 1,000 possible models. If $p=20$ there are over one million possibilities. Best subset becomes computationally infeasible around $p=40$, even with extremely fast modern computers.  

## Stepwise Selection  
Best subset selection cannot be applied with very large $p$. Best subset selection may also suffer from statistical problems when $p$ is large. The larger the search space, the higher the chance of finding models that look good on training data, even though they might not have any predictive power on future data. Thus, an enourmous search space can lead to overfitting and high variance of the coefficient estimates.  

*Stepwise* methods explore a far more restricted set of models.

### Forward Stepwise Selection  
*Forward stepwise selection* begins with a model containing no predictors, and then adds predictors to the model that gives the greatest *additional* improvement, one-at-a-time until all predictors are in the model.  

---
**Algorithm 6.2**: *Forward stepwise selection  

  1. Let $M_0$ denote the *null* model, which contains no predictors.  
  <br>
  2. For $k=0, ..., p-1$:  
    a. Consider all $p-k$ models that augment the predictors in $M_k$ with one additional predictor.  
    b. Choose the *best* among these $p-k$ models, and call it $M_{k+1}$. Here *best* is defined as having the smallest RSS or highest $R^2$.  
  <br>
  3. Select a single best model from among $M_0, ..., M_p$ using cross validated prediction error, $C_p$ (AIC), BIC, or adjusted $R^2$  
  
---  

Forward stepwise selection involves fitting $1 + \sum_{k=0}^{p-1}(p-k) = 1 + p(p+1)/2$ instead of $2^p$ models for best subset selection. When $p=20$, best subset selection requires fitting 1,048,576 models, wheras forward stepwise selection requires fitting only 211 models!  

While forward stepwise selection's computational advantages are clear, it is not guaranteed to find the best possible model out of all $2^p$ possible models. Consider a data set with $p=3$ predictors; the best possible one-variable model icludes $X_1$ and the best possible two-variable model contains $X_2$ and $X_3$. Forward selection will fail to select the best possible two-variable model, because $M_1$ will contain $X_1$, so $M_2$ must contain $X_1$ as well.  

Forward stepwise selection can be applied to high-dimensional data sets (e.g. $n<p$), however it is only possible to construct submodels $M_0, M_{n-1}$ since least squares will not yield a unique solution if $p \ge n$.

### Backward Stepwise Selection  
Unlike forward stepwise selection, backward stepwise selection begins with the full least squares model containing all $p$ predictors, and then iteratively removes the least useful predictor one-at-a-time.  

---  
**Algorithm 6.3**: *Backward Stepwise Selection*  

  1. Let $M_p$ denote the *full* model, which contains all $p$ predictors.  
  <br>
  2. For $k = p, p-1, ..., 1$:  
    a. Consider all $k$ models that contain all but one of the predictors in $M_k$, for a total of $k-1$ predictors.  
    b. Choose the *best* among these $k$ models, and call it $M_{k-1}$. here *best* is defined as having smallest RSS or highest $R^2$.  
  <br>
  3. Select a single best model from among $M_0, ..., M_p$ using cross validated prediction error, $C_p$ (AIC), BIC, or adjusted $R^2$.  
  
---  

Backward stepwise selection searches through $1+p(p-1)/2$ models as in forward selection so it can be applied when $p$ is too large to apply best subset selection. Also like forward stepwise selection, backward stepwise selection is not guaranteed to yield the *best* model containing a subset of the $p$ predictors.  

Backward selection requires that the number of samples $n$ is larger than the number of variables $p$ (so that the full model can be fit). In contrast, forward selection can be used even when $n<p$, and so is the only viable subset method when $p$ is very large.  

### Hybrid Approaches  
Hybrid versions of forward and backward stepwise selection are available, in which variables are added to the model sequentially, in analogy to forward selection. However, after adding each new variable, the method may also remove any variables that no longer provide an improvement in the model fit. Such an approach attempts to more closely mimic best sub- set selection while retaining the computational advantages of forward and backward stepwise selection

## Choosing the Optimal Model  
Since the model containing all $p$ predictors will always have the smallest RSS and the largest $R^2$ (since they are related to *training* error), we need to select the best model with respect to *test* error. There are two common approaches:  

  1. We can indirectly estimate the test error by making an *adjustment* to the training error to account for the bias due to overfitting.  
  
  2. We can *directly* estimate the test error, using either a validation set approach or a cross-validation approach. 

### $C_p$, AIC, BIC, and Adjusted $R^2$  
The training set MSE is generally an underestimate of the test MSE (recall MSE = RSS/$n$) because when we fit a model to the training data using least squares, we specifically estimate the regression coefficients such that the training RSS (but not the test RSS) is minimized. In particular, the training error will decrease as more variables are included in the model but the test error may not. Therefore, training set RSS and training set $R^2$ cannot be used to select from among the set of models with a different number of variables.  

A number of techniques for *adjusting* the training error for the model size are available. These approaches can be used to select among a set of models with different number of variables.  

For a fitted least squares model containing $d$ predictors, the $C_p$ estimate of the test MSE is computed using the equation  

$$C_p=\frac{1}{n}(RSS+2d\widehat{\sigma}^2)$$  

where $\widehat{\sigma}^2$ is an estimate of the variance of the error $\epsilon$ associated with each response measurement. Typically $\widehat{\sigma}^2$ is estimated using the full model containing all predictors. Essentially, the $C_p$ statistic adds a penalty of $2d\widehat{\sigma}^2$ to the training RSS in order to adjust for the fact that the training error tends to underestimate the test error. The penalty increases as the number of predictors in the model increases, which is intended to adjust for the corresponding decrease in RSS. It can be shown that if $\widehat{\sigma}^2$ is an unbiased estimator of $\sigma^2$, then $C_p$ is an unbiased estimate of test MSE.  

The *Akaike information criterion* (AIC) is defined for a large class of models fit by maximum likelihood. In the case of linear regression with Gaussian errors, maximum likelihood and least squares are the same thing. In this case AIC is given by  

$$AIC=\frac{1}{n\widehat{\sigma}^2}(RSS + 2d\widehat{\sigma}^2)$$  

where, for simplicity, an additive constant has been omitted. For least squares, $C_p$ and AIC are proportional to each other.  

The *Bayesian information criterion* is derived from a Bayesian point of view, but looks similar to $C_p$ and AIC. For the least squares model with $d$ predictors, the BIC is, up to irrelevant constants, given by  

$$BIC = \frac{1}{n\widehat{\sigma}^2}(RSS + \text{log}(n)d\widehat{\sigma}^2)$$  

Notice that BIC replaces the $2d\widehat{\sigma}^2$ with a $\text{log}(n)d\widehat{\sigma}^2$. Since $\text{log}(n) > 2$ for any $n>7$, the BIC statistic generally places a heavier penalty on models with many variables, and hence results in the selection of smaller models than $C_p$.  

The adjusted $R^2$ statistic is another popular approach for selecting among a set of models that contain different numbers of variables. The usual $R^2$ is defined as $1-RSS/TSS$ where $TSS=\sum(y_i-\bar{y})^2$ is the *total sum of squares* for the response. Since RSS always decreases as more variables are added to the model, the $R^2$ always increases as more variables are added. For a least squares model with $d$ variables, the adjusted $R^2$ statistic is calculated as  

$$\text{Adjusted }R^2=1-\frac{RSS/(n-d-1)}{TSS/(n-1)}$$  

Unlike $C_p$, AIC, and BIC, for which a small value indicates a model with low test error, a large value of adjusted $R^2$ indicates a model with a small test error. Maximizing the adjusted $R^2$ is equivalent to minimizing $\frac{RSS}{n-d-1}$, which may increase or decrease as the number of variables in the model increases, due to the presence of $d$ in the denominator.  

The intuition is that once all of the correct variables have been included in the model, adding additional *noise* variables will lead to only a very small decrease in RSS. However, adding noise variables leads to an increase in $d$, which will lead to an increase in $\frac{RSS}{n-d-1}$, and consequently a decrease in the adjusted $R^2$. Unlike the $R^2$ statistic, the adjusted $R^2$ statistic *pays a price* for the inclusion of unnecessary variables in the model.

### Validation and Cross-Validation  
We can also directly estimate the test error using the validation set and cross-validation methods. We can compute the validation set error or the cross-validation error for each model under consideration, and then select the model for which the resulting estimated test error is smalles. This has an advantage relative to $C_p$, AIC, BIC, and adjusted $R^2$, in that it provides a direct estimate of test error and makes less assumptions about the true underlying model. It can also be used in a wider range of model selection tasks, even in cases where it is hard to pinpoint the model degrees of freedom or hard to estimate the error variance $\sigma^2$.  

Because the estimated test error of models which include an increasing number of variables are quite similar, we can select a model using the *one-standard-error rule*. We first calculate the standard error of the estimated test MSE for each model size, and then select the smallest model for which the estimated test error is within one standard error of the lowest point on the curve. The rationale is that if a set of models appear to be more or less equally qood, then we might as well choose the simplest method (least number of predictors).  

# Shrinkage Methods  
As an alternative to fitting a linear model with least squares, we can fit a model containing all $p$ predictors using a technique that *constrains* or *regularizes* the coefficient estimates, ore equivalently, that *shrinks* the coefficients towards zero. It turns out that shrinking the coefficient estimates can significantly reduce their variance. The best-known techniques for shrinking the regression coefficients towards zero are *ridge regression * and the *lasso*.  

## Ridge Regression  
The least squares fitting procedure estimates $\beta_0, \beta_1, ..., \beta_p$ using the values that minimize  

$$RSS=\sum_{i=1}^n\biggl(y_i - \beta_0 -\sum_{j=1}^p\beta_jx_{ij}\biggr)^2$$  

*Ridge Regression* is very similar to least squares, except that the coefficients are estimated by minimizing a slightly different quantitiy. In particular, the ridge regression coefficient estimates $\widehat{\beta}^R$ are the values that minimize  

$$\sum_{i=1}^n\biggl(y_i-\beta_0-\sum_{j=1}^p\beta_jx_{ij}\biggr)^2 + \lambda\sum_{j=1}^p\beta_j^2=RSS+\lambda\sum_{j=1}^p\beta_j^2$$  

where $\lambda \ge 0$ is a *tuning parameter*, to be determined separately. As with least squares, ridge regression seeks coefficient estimates that fit the data well, by making the RSS small. However, the second term $\lambda\sum_j\beta_j^2$, called the *shrinkage penalty*, is small when $\beta_1, ..., \beta_p$ are close to zero, and so it has the effect of *shrinking* the estimates of $\beta_j$ towards zero. $\lambda$ serves to control the relative impact of these two terms on the regression coefficient estimates. When $\lambda=0$, the penalty term has no effect, and ridge regression will produce the least squares estimates. However, as $\lambda \to \infty$, the impact of the shrinkage penalty grows, and the ridge regression coefficient estimates will approach zero. Ridge regression will produce a different set of coefficient estimates, $\widehat{\beta}_{\lambda}^R$, for each value of $\lambda$.  

Notice that the shrinkage penalty is applied to $\beta_1, ..., \beta_p$ but not to the intercept $\beta_0$. We want to shrink the estimated association of each variable with the response; however, we do not want to shrink the intercept, which is simply a measure of the mean value of the response when $x_{i1}=x_{i2}=...=x_{ip}=0$.